In [1]:
!pip install azure-cognitiveservices-language-textanalytics pandas

     ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
     ---------------------------------------- 44.8/44.8 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/85.4 kB ? eta -:--:--
     ---------------------------------------- 85.4/85.4 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/151.7 kB ? eta -:--:--
     ---------------------------------------- 151.7/151.7 kB ? eta 0:00:00


In [19]:
import json
import pandas as pd
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials
import time

# Azure Cognitive Services configuration
subscription_key = '9e2da6a0d11c4fb28ef9b06d4cd1b2df'
endpoint = 'https://dataforgood.cognitiveservices.azure.com/'
location = 'useast2'  # Add your Azure Translator service location

translator_client = TextAnalyticsClient(endpoint=endpoint, credentials=CognitiveServicesCredentials(subscription_key))

# Function to detect language and translate
def detect_and_translate(text):
    response = translator_client.detect_language(text)
    if response.primary_language.name != 'en':
        translation = translator_client.translate(text, target_language='en')
        return translation
    return text

# Read JSON file
with open('transcripts.json', 'r') as json_file:
    data = json.loads(json_file.read())

# Create an empty list to store the translated data
translated_data = []

# Initialize variables for rate limiting
characters_translated = 0
characters_limit = 50000
characters_per_request = 5000
request_interval = 12  # 60 minutes / 5 requests per minute (to stay within hourly quota)

# Serial numbers to process
serial_numbers_to_process = [76, 157, 127, 201, 478]

# Process each entry in the JSON file
for serial_no, text in data.items():
    if int(serial_no) not in serial_numbers_to_process:
        continue  # Skip if serial number not in the list

    # Check if text is in English, and if not, translate it
    text_length = len(text)
    if text_length + characters_translated > characters_limit:
        time.sleep(request_interval)
        characters_translated = 0  # Reset characters translated

    translated_text = detect_and_translate(text)
    characters_translated += text_length

    translated_data.append({'Serial no': serial_no, 'text': translated_text})

# Convert the translated data to a DataFrame
df = pd.DataFrame(translated_data)

# Save the DataFrame to a CSV file
df.to_csv('translated_transcripts.csv', index=False)

print("Translation and saving completed.")


ErrorResponseException: Operation returned an invalid status code 'Access Denied'